# 1. lstm
* 用pytorch做listm模型
* 将长度变化的句子输入lism 得到隐藏层和输出成，当做编码的结果。
* 如果效果不好的话可以试一下，crf?attention?
        
# 2.  现在的问题。

* 预测的时候还要重新计算词向量
* 加入embeding层
        

In [1]:
import jieba 
import numpy as np
import torch
from torch import nn
from sklearn.metrics import f1_score
import torch.nn.functional as F
from batch_size import datasets
from torch.utils.data import Dataset, DataLoader
from preprocess import pre_train
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd
import config as c
import pickle

F:\app\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
load_word=pre_train()
data=load_word.load_word()

## 1.1 word_embeding
如果需要自己训练embeding
如果内存真的放不下，最好用embeding

In [3]:
def is_chinese(uchar):         
    if '\u4e00' <= uchar<='\u9fff':
        return True
    else:
        return False

def jiebaclearText(data,stopwords_path=r'../data/stop_words.txt'):
    '''保留所有的汉字。
    '''
    for sents in data['q1']:
        '''句子'''
        for w in sents:
            '''一个单词'''
            if not is_chinese(w):
                sents.remove(w)
    for sents in data['q2']:
        '''句子'''
        for w in sents:
            '''一个单词'''
            if not is_chinese(w):
                sents.remove(w)
                
    return data
data=jiebaclearText(data)

In [4]:
data['q1'][0]

['怎么', '更改', '花', '呗', '手机号码']

### 1.2获得词汇
vocab

In [5]:

def get_vocab():
    vocab=set()
    for i in data['q1']:
        for j in i:
            vocab.add(j)
    for i in data['q2']:
        for j in i:
            vocab.add(j)
    return list(vocab)
vocab=get_vocab()


* 只能读取，除非我又改了切词的方式

In [6]:
word2index=pickle.load(open('../data/word2index-3.pkl','rb'))

* vocab to index

In [7]:
'花呗' in word2index

False

In [ ]:
word2index={i:index for index,i in enumerate(vocab)}
pickle.dump(word2index,open('word2index-2.pkl','wb'),2)
pickle.dump(word2index,open('word2index-3.pkl','wb'),3)

### 2. lstm得到隐藏层

* 我希望把每个句子（seq_len）内单词对应的矢量按照顺序输入之后得到一个维度相同的隐藏层。
* 有两种情况，
    * 我把两个句子合并输入，直接得到x,y
    * 我把两个句子输入之后得到输出，然后在把输出合并，得到x,y

### 保存模型

In [ ]:
torch.save(model.state_dict(),'../model/lstm_v2_100_state_ditc.pkl')

In [ ]:
model.load_state_dict(torch.load('../model/lstm_v2_100_state_ditc.pkl'))

# 第二种
 * 把两个的输出链接起来然后再预测


# FEED DATA

In [8]:
#feed data
def get_one_word_for_concat(word):
    words=np.zeros((len(word),))-1
    for i,w in enumerate(word):
        if w in word2index:
            words[i]=word2index[w]
    return words

In [9]:
#feed data2
def get_data_lstm_concat(idx):
    '''
    就是把数据传入,batch_size=1
    seq_len ,word_dim
    '''
    idx=idx[0]
    word1=get_one_word_for_concat(data['q1'][idx])
    word2=get_one_word_for_concat(data['q2'][idx])
    target=np.array([data['target'][idx]])    
    return word1,word2,target


In [12]:
x0,x1,y=get_data_lstm_concat([1])
x0,x1,y

(array([11696., 11322., 11622., 11342.,  4697.,  7137., 13114.,  6344.,
         5996.,  6344.]),
 array([10902.,  3022.,  3639., 11342.,  4697.,  6240.]),
 array([0], dtype=int64))

# model
* Nishant Nikhil (i.nishantnikhil@gmail.com)
* An implementation of the Deep Semantic Similarity Model (DSSM) found in [1].
* [1] Shen, Y., He, X., Gao, J., Deng, L., and Mesnil, G. 2014. A latent semantic odel with convolutional-pooling structure for information retrieval. In CIKM, pp. 101-110.http://research.microsoft.com/pubs/226585/cikm2014_cdssm_final.pdf
* [2] http://research.microsoft.com/en-us/projects/dssm/
* [3] http://research.microsoft.com/pubs/238873/wsdm2015.v3.pdf

In [ ]:
LETTER_GRAM_SIZE = 3 # See section 3.2.
WINDOW_SIZE = 3 # See section 3.2.
TOTAL_LETTER_GRAMS = int(3 * 1e4) # Determined from data. See section 3.2.
WORD_DEPTH = WINDOW_SIZE * TOTAL_LETTER_GRAMS # See equation (1).
# Uncomment it, if testing
# WORD_DEPTH = 1000
K = 300 # Dimensionality of the max-pooling layer. See section 3.4.
L = 128 # Dimensionality of latent semantic space. See section 3.5.
J = 4 # Number of random unclicked documents serving as negative examples for a query. See section 4.
FILTER_LENGTH = 1 # We only consider one time step for convolutions.


def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

class CDSSM(nn.Module):
    def __init__(self):
        super(CDSSM, self).__init__()
        # layers for query
        self.query_conv = nn.Conv1d(WORD_DEPTH, K, FILTER_LENGTH)
        self.query_sem = nn.Linear(K, L)
        # layers for docs
        self.doc_conv = nn.Conv1d(WORD_DEPTH, K, FILTER_LENGTH)
        self.doc_sem = nn.Linear(K, L)
        # learning gamma
        self.learn_gamma = nn.Conv1d(1, 1, 1)
    def forward(self, q, pos, negs):
        # Query model. The paper uses separate neural nets for queries and documents (see section 5.2).
        # To make it compatible with Conv layer we reshape it to: (batch_size, WORD_DEPTH, query_len)
        q = q.transpose(1,2)
        # In this step, we transform each word vector with WORD_DEPTH dimensions into its
        # convolved representation with K dimensions. K is the number of kernels/filters
        # being used in the operation. Essentially, the operation is taking the dot product
        # of a single weight matrix (W_c) with each of the word vectors (l_t) from the
        # query matrix (l_Q), adding a bias vector (b_c), and then applying the tanh activation.
        # That is, h_Q = tanh(W_c • l_Q + b_c). Note: the paper does not include bias units.
        q_c = F.tanh(self.query_conv(q))
        # Next, we apply a max-pooling layer to the convolved query matrix.
        q_k = kmax_pooling(q_c, 2, 1)
        q_k = q_k.transpose(1,2)
        # In this step, we generate the semantic vector represenation of the query. This
        # is a standard neural network dense layer, i.e., y = tanh(W_s • v + b_s). Again,
        # the paper does not include bias units.
        q_s = F.tanh(self.query_sem(q_k))
        q_s = q_s.resize(L)
        # # The document equivalent of the above query model for positive document
        pos = pos.transpose(1,2)
        pos_c = F.tanh(self.doc_conv(pos))
        pos_k = kmax_pooling(pos_c, 2, 1)
        pos_k = pos_k.transpose(1,2)
        pos_s = F.tanh(self.doc_sem(pos_k))
        pos_s = pos_s.resize(L)
        # # The document equivalent of the above query model for negative documents
        negs = [neg.transpose(1,2) for neg in negs]
        neg_cs = [F.tanh(self.doc_conv(neg)) for neg in negs]
        neg_ks = [kmax_pooling(neg_c, 2, 1) for neg_c in neg_cs]
        neg_ks = [neg_k.transpose(1,2) for neg_k in neg_ks]
        neg_ss = [F.tanh(self.doc_sem(neg_k)) for neg_k in neg_ks]
        neg_ss = [neg_s.resize(L) for neg_s in neg_ss]
        # Now let us calculates the cosine similarity between the semantic representations of
        # a queries and documents
        # dots[0] is the dot-product for positive document, this is necessary to remember
        # because we set the target label accordingly
        dots = [q_s.dot(pos_s)]
        dots = dots + [q_s.dot(neg_s) for neg_s in neg_ss]
        # dots is a list as of now, lets convert it to torch variable
        dots = torch.stack(dots)
        # In this step, we multiply each dot product value by gamma. In the paper, gamma is
        # described as a smoothing factor for the softmax function, and it's set empirically
        # on a held-out data set. We're going to learn gamma's value by pretending it's
        # a single 1 x 1 kernel.
        with_gamma = self.learn_gamma(dots.resize(J+1, 1, 1))
        # Finally, we use the softmax function to calculate P(D+|Q).
        prob = F.softmax(with_gamma)
        return prob

model = CDSSM()

In [ ]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:






# Build a random data set.
import numpy as np
sample_size = 10
l_Qs = []
pos_l_Ds = []

(query_len, doc_len) = (5, 100)

for i in range(sample_size):
    query_len = np.random.randint(1, 10)
    l_Q = np.random.rand(1, query_len, WORD_DEPTH)
    l_Qs.append(l_Q)
    
    doc_len = np.random.randint(50, 500)
    l_D = np.random.rand(1, doc_len, WORD_DEPTH)
    pos_l_Ds.append(l_D)

neg_l_Ds = [[] for j in range(J)]
for i in range(sample_size):
    possibilities = list(range(sample_size))
    possibilities.remove(i)
    negatives = np.random.choice(possibilities, J, replace = False)
    for j in range(J):
        negative = negatives[j]
        neg_l_Ds[j].append(pos_l_Ds[negative])

# Till now, we have made a complete numpy dataset
# Now let's convert the numpy variables to torch Variable

for i in range(len(l_Qs)):
    l_Qs[i] = Variable(torch.from_numpy(l_Qs[i]).float())
    pos_l_Ds[i] = Variable(torch.from_numpy(pos_l_Ds[i]).float())
    for j in range(J):
        neg_l_Ds[j][i] = Variable(torch.from_numpy(neg_l_Ds[j][i]).float())


# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

# output variable, remember the cosine similarity with positive doc was at 0th index
y = np.ndarray(1)
# CrossEntropyLoss expects only the index as a long tensor
y[0] = 0
y = Variable(torch.from_numpy(y).long())

for i in range(sample_size):
    y_pred = model(l_Qs[i], pos_l_Ds[i], [neg_l_Ds[j][i] for j in range(J)])
    loss = criterion(y_pred.resize(1,J+1), y)
    print (i, loss.data[0])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


* train

In [ ]:
y_pred0=[]
y=[]
model.eval()
with torch.no_grad():
    for i in range(len(data['index'])):
        train_x,train_y=get_data_lstm([i])
        inputs=Variable(torch.LongTensor(train_x))
        labels=Variable(torch.tensor(train_y).long())
        y_pred0.append(model(inputs))
        y.append(labels)    
        
y_pred=[i.argmax() for i in y_pred]
print('f1_score',f1_score(y,y_pred))